# Capstone Project: Kaggle Submission Book

In this project, the book acts as supplementary book for Kaggle Submissions because I've switched models to run the final Kaggle files because the first batches of results performed poorly on the Kaggle data

In [7]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pickle
import time

from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import roc_curve
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

from collections import Counter
from operator import itemgetter

# magic word for producing visualizations in notebook
%matplotlib inline

## Part 0.4: Clean_data Function

In [11]:
def clean_data(df1, df2):
    """
    Perform feature trimming, re-encoding, and engineering for demographics
    data
    
    INPUT: 
        df1: Customer demographics DataFrame
        df2: Unknowns DataFrame
    OUTPUT: Trimmed and cleaned demographics DataFrame
    """
    for col in df1.columns:
        if df1[col].dtype == np.int64:
            df1[col] = df1[col].astype(np.float64)
        
    # convert missing value codes into NaNs, ...
    for row in df2['attribute']:
        if row in df1.columns:
            na_map = df2.loc[df2['attribute'] == row, 'missing_or_unknown'].iloc[0]
            na_idx = df1.loc[:, row].isin(na_map)
            df1.loc[na_idx, row] = np.NaN
        else:
            continue
            
    # remove selected columns, ...
    # These columns were the ones above that had
    # > 200000 missing values in the azdias dataframe
    cols_to_drop = ['EINGEFUEGT_AM','CAMEO_DEUG_2015','CAMEO_INTL_2015', 'AGER_TYP', 'ALTER_HH', 'ALTER_KIND1', 'ALTER_KIND2', 'ALTER_KIND3',
       'ALTER_KIND4', 'ALTERSKATEGORIE_FEIN', 'D19_BANKEN_ANZ_12',
       'D19_BANKEN_ANZ_24', 'D19_BANKEN_DATUM', 'D19_BANKEN_OFFLINE_DATUM',
       'D19_BANKEN_ONLINE_DATUM', 'D19_BANKEN_ONLINE_QUOTE_12',
       'D19_GESAMT_ANZ_12', 'D19_GESAMT_ANZ_24', 'D19_GESAMT_DATUM',
       'D19_GESAMT_OFFLINE_DATUM', 'D19_GESAMT_ONLINE_DATUM',
       'D19_GESAMT_ONLINE_QUOTE_12', 'D19_KONSUMTYP',
       'D19_LETZTER_KAUF_BRANCHE', 'D19_LOTTO', 'D19_SOZIALES',
       'D19_TELKO_ANZ_12', 'D19_TELKO_ANZ_24', 'D19_TELKO_DATUM',
       'D19_TELKO_OFFLINE_DATUM', 'D19_TELKO_ONLINE_DATUM',
       'D19_TELKO_ONLINE_QUOTE_12', 'D19_VERSAND_ANZ_12', 'D19_VERSAND_ANZ_24',
       'D19_VERSAND_DATUM', 'D19_VERSAND_OFFLINE_DATUM',
       'D19_VERSAND_ONLINE_DATUM', 'D19_VERSAND_ONLINE_QUOTE_12',
       'D19_VERSI_ANZ_12', 'D19_VERSI_ANZ_24', 'D19_VERSI_ONLINE_QUOTE_12',
       'EXTSEL992', 'KBA05_ANTG1', 'KBA05_ANTG2', 'KBA05_ANTG3', 'KBA05_ANTG4',
       'KBA05_BAUMAX', 'KBA05_MAXVORB', 'KK_KUNDENTYP', 'TITEL_KZ']
    df1.drop(cols_to_drop, axis=1, inplace = True)
    
    # remove selected rows, ...
    # These are rows which have >25 missing values
    #row_na = df1.shape[1] - df1.count(axis = 1)
    #rows_to_drop = df1.index[row_na > 10]
    #df1.drop(rows_to_drop, axis=0, inplace = True)
    
    # select, re-encode, and engineer column values
    df1['OST_WEST_KZ'].replace(['W', 'O'], [1, 0], inplace=True)

    cols_to_dummy = ['CJT_GESAMTTYP', 'FINANZTYP', 'GEBAEUDETYP', 'GEBAEUDETYP_RASTER', 'HEALTH_TYP', 
                    'KBA05_HERSTTEMP', 'KBA05_MAXHERST', 'KBA05_MODTEMP', 'LP_FAMILIE_GROB', 'LP_STATUS_GROB',
                    'NATIONALITAET_KZ', 'SHOPPER_TYP', 'VERS_TYP']
    
    for col in cols_to_dummy:
        dummy = pd.get_dummies(df1[col], prefix = col)
        df1 = pd.concat([df1, dummy], axis = 1)
        
    df1.drop(cols_to_dummy, axis=1, inplace = True)
    
    cols_to_drop = ['CAMEO_DEU_2015', 'GFK_URLAUBERTYP', 'LP_FAMILIE_FEIN', 'LP_LEBENSPHASE_FEIN', 
                   'LP_LEBENSPHASE_GROB', 'LP_STATUS_FEIN', 'PRAEGENDE_JUGENDJAHRE']
    
    df1.drop(cols_to_drop, axis=1, inplace = True)
    
    # Return the cleaned dataframe.
    return df1

## Load & Clean the training data

In [23]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')
mailout_train.head()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,RESPONSE,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,1763,2,1.0,8.0,NaN,NaN,NaN,NaN,8.0,15.0,...,5.0,2.0,1.0,6.0,9.0,3.0,3,0,2,4
1,1771,1,4.0,13.0,NaN,NaN,NaN,NaN,13.0,1.0,...,1.0,2.0,1.0,4.0,9.0,7.0,1,0,2,3
2,1776,1,1.0,9.0,NaN,NaN,NaN,NaN,7.0,0.0,...,6.0,4.0,2.0,NaN,9.0,2.0,3,0,1,4
3,1460,2,1.0,6.0,NaN,NaN,NaN,NaN,6.0,4.0,...,8.0,11.0,11.0,6.0,9.0,1.0,3,0,2,4
4,1783,2,1.0,9.0,NaN,NaN,NaN,NaN,9.0,53.0,...,2.0,2.0,1.0,6.0,9.0,3.0,3,0,1,3


In [9]:
# Reload cleaned azdias object as saved after above analysis (may need to rerun imports)
unknowns = pickle.load(open("unknowns.pickle", "rb"))

In [24]:
# Clean the training data (may need to rerun cell with cleaning function, loading unknowns.pickle)
mailout_train = clean_data(mailout_train, unknowns)

In [25]:
mailout_train.head()

,LNR,AKT_DAT_KL,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_KINDER,ANZ_PERSONEN,ANZ_STATISTISCHE_HAUSHALTE,ANZ_TITEL,ARBEIT,BALLRAUM,...,LP_STATUS_GROB_5.0,NATIONALITAET_KZ_1.0,NATIONALITAET_KZ_2.0,NATIONALITAET_KZ_3.0,SHOPPER_TYP_0.0,SHOPPER_TYP_1.0,SHOPPER_TYP_2.0,SHOPPER_TYP_3.0,VERS_TYP_1.0,VERS_TYP_2.0
0,1763.0,1.0,15.0,0.0,0.0,1.0,13.0,0.0,3.0,5.0,...,0,1,0,0,0,0,0,1,0,1
1,1771.0,4.0,1.0,0.0,0.0,2.0,1.0,0.0,2.0,5.0,...,0,0,1,0,0,0,1,0,1,0
2,1776.0,1.0,0.0,NaN,0.0,0.0,1.0,0.0,4.0,1.0,...,1,1,0,0,0,0,0,1,1,0
3,1460.0,1.0,4.0,0.0,0.0,2.0,4.0,0.0,4.0,2.0,...,0,1,0,0,0,1,0,0,0,1
4,1783.0,1.0,53.0,0.0,0.0,1.0,44.0,0.0,3.0,4.0,...,0,1,0,0,0,1,0,0,1,0


In [26]:
# Define the labels (y) to be the response column, and the rest of the columns to be X
y = mailout_train['RESPONSE']
del mailout_train['RESPONSE']
X = mailout_train
y.shape, X.shape

((42962,), (42962, 358))

In [27]:
# Obtain the vale counts for the response column
y.value_counts()

0.0    42430
1.0      532
Name: RESPONSE, dtype: int64

In [28]:
# Print first few responses to ensre they are in an expected form
y.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: RESPONSE, dtype: float64

In [29]:
# Perform imputation on train df
imputer = Imputer()
X = pd.DataFrame(imputer.fit_transform(X), columns = X.columns)
X.head()

,LNR,AKT_DAT_KL,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_KINDER,ANZ_PERSONEN,ANZ_STATISTISCHE_HAUSHALTE,ANZ_TITEL,ARBEIT,BALLRAUM,...,LP_STATUS_GROB_5.0,NATIONALITAET_KZ_1.0,NATIONALITAET_KZ_2.0,NATIONALITAET_KZ_3.0,SHOPPER_TYP_0.0,SHOPPER_TYP_1.0,SHOPPER_TYP_2.0,SHOPPER_TYP_3.0,VERS_TYP_1.0,VERS_TYP_2.0
0,1763.0,1.0,15.0,0.000000,0.0,1.0,13.0,0.0,3.0,5.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,1771.0,4.0,1.0,0.000000,0.0,2.0,1.0,0.0,2.0,5.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,1776.0,1.0,0.0,0.049574,0.0,0.0,1.0,0.0,4.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,1460.0,1.0,4.0,0.000000,0.0,2.0,4.0,0.0,4.0,2.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,1783.0,1.0,53.0,0.000000,0.0,1.0,44.0,0.0,3.0,4.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [30]:
# Apply feature scaling to the train df
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)
X.head()

,LNR,AKT_DAT_KL,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_KINDER,ANZ_PERSONEN,ANZ_STATISTISCHE_HAUSHALTE,ANZ_TITEL,ARBEIT,BALLRAUM,...,LP_STATUS_GROB_5.0,NATIONALITAET_KZ_1.0,NATIONALITAET_KZ_2.0,NATIONALITAET_KZ_3.0,SHOPPER_TYP_0.0,SHOPPER_TYP_1.0,SHOPPER_TYP_2.0,SHOPPER_TYP_3.0,VERS_TYP_1.0,VERS_TYP_2.0
0,-1.656309,-0.329514,0.604874,-0.145,-0.245521,-0.818214,0.518647,-0.10013,-0.048327,0.374223,...,-0.519519,0.498021,-0.130555,-0.108733,-0.404226,-0.525800,-0.419113,1.453138,-0.871293,1.234287
1,-1.655987,1.552561,-0.416145,-0.145,-0.245521,-0.013746,-0.406937,-0.10013,-1.117178,0.374223,...,-0.519519,-2.007949,7.659598,-0.108733,-0.404226,-0.525800,2.385989,-0.688166,1.147719,-0.810184
2,-1.655785,-0.329514,-0.489075,0.000,-0.245521,-1.622682,-0.406937,-0.10013,1.020523,-1.639828,...,1.924856,0.498021,-0.130555,-0.108733,-0.404226,-0.525800,-0.419113,1.453138,1.147719,-0.810184
3,-1.668538,-0.329514,-0.197355,-0.145,-0.245521,-0.013746,-0.175541,-0.10013,1.020523,-1.136315,...,-0.519519,0.498021,-0.130555,-0.108733,-0.404226,1.901864,-0.419113,-0.688166,-0.871293,1.234287
4,-1.655502,-0.329514,3.376210,-0.145,-0.245521,-0.818214,2.909739,-0.10013,-0.048327,-0.129290,...,-0.519519,0.498021,-0.130555,-0.108733,-0.404226,1.901864,-0.419113,-0.688166,1.147719,-0.810184


## Part 2.1: Set up a classifier runner & ROC score printer

In [31]:
def classifier_roc(clf, param_grid, X=X, y=y):
    """
    Fit a classifier using GridSearchCV and calculates ROC AUC
    
    INPUT:
    - clf (classifier): classifier to fit
    - param_grid (dict): classifier parameters used with GridSearchCV
    - X (DataFrame): features of the training dataframe
    - y (DataFrame): labels of the training dataframe
            
    OUTPUT:
    - classifier: fitted classifier
    - prints elapsed time and ROX AUC
    """
    
    # cv uses StratifiedKFold
    # scoring roc_auc available as parameter
    
    start = time.time()
    
    grid = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='roc_auc', cv=5)
    grid.fit(X, y)
    
    end = time.time()
    print(end-start)

    print(grid.best_score_)
    
    return grid.best_estimator_

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter. If you're one of the top performers, you may have the chance to be contacted by a hiring manager from Arvato or Bertelsmann for an interview!

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

## Load & Preprocess Data

In [8]:
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
mailout_test = clean_data(mailout_test, unknowns)
mailout_test.head()

,LNR,AKT_DAT_KL,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_KINDER,ANZ_PERSONEN,ANZ_STATISTISCHE_HAUSHALTE,ANZ_TITEL,ARBEIT,BALLRAUM,...,LP_STATUS_GROB_5.0,NATIONALITAET_KZ_1.0,NATIONALITAET_KZ_2.0,NATIONALITAET_KZ_3.0,SHOPPER_TYP_0.0,SHOPPER_TYP_1.0,SHOPPER_TYP_2.0,SHOPPER_TYP_3.0,VERS_TYP_1.0,VERS_TYP_2.0
0,1754.0,1.0,2.0,0.0,0.0,2.0,2.0,0.0,3.0,6.0,...,1,1,0,0,0,0,0,1,1,0
1,1770.0,1.0,20.0,0.0,0.0,1.0,21.0,0.0,4.0,7.0,...,0,1,0,0,0,0,1,0,1,0
2,1465.0,9.0,2.0,0.0,0.0,4.0,2.0,0.0,4.0,1.0,...,1,1,0,0,0,0,0,1,1,0
3,1470.0,7.0,1.0,0.0,0.0,0.0,1.0,0.0,4.0,1.0,...,0,1,0,0,0,0,0,1,0,1
4,1478.0,1.0,1.0,0.0,0.0,4.0,1.0,0.0,3.0,6.0,...,0,1,0,0,0,0,0,1,1,0


In [14]:
mailout_test.shape

(42833, 358)

In [15]:
# Copy LNR column for later when generating Kaggle submission csv 
test_LNR = mailout_test['LNR']

In [20]:
# Perform imputation on train df
imputer = Imputer()
mailout_test = pd.DataFrame(imputer.fit_transform(mailout_test), columns = mailout_test.columns)
mailout_test.head()

,LNR,AKT_DAT_KL,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_KINDER,ANZ_PERSONEN,ANZ_STATISTISCHE_HAUSHALTE,ANZ_TITEL,ARBEIT,BALLRAUM,...,LP_STATUS_GROB_5.0,NATIONALITAET_KZ_1.0,NATIONALITAET_KZ_2.0,NATIONALITAET_KZ_3.0,SHOPPER_TYP_0.0,SHOPPER_TYP_1.0,SHOPPER_TYP_2.0,SHOPPER_TYP_3.0,VERS_TYP_1.0,VERS_TYP_2.0
0,1754.0,1.0,2.0,0.0,0.0,2.0,2.0,0.0,3.0,6.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,1770.0,1.0,20.0,0.0,0.0,1.0,21.0,0.0,4.0,7.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,1465.0,9.0,2.0,0.0,0.0,4.0,2.0,0.0,4.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,1470.0,7.0,1.0,0.0,0.0,0.0,1.0,0.0,4.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,1478.0,1.0,1.0,0.0,0.0,4.0,1.0,0.0,3.0,6.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [22]:
# Apply feature scaling to the test df
scaler = StandardScaler()
mailout_test = pd.DataFrame(scaler.fit_transform(mailout_test), columns = mailout_test.columns)
mailout_test.head()


,LNR,AKT_DAT_KL,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_KINDER,ANZ_PERSONEN,ANZ_STATISTISCHE_HAUSHALTE,ANZ_TITEL,ARBEIT,BALLRAUM,...,LP_STATUS_GROB_5.0,NATIONALITAET_KZ_1.0,NATIONALITAET_KZ_2.0,NATIONALITAET_KZ_3.0,SHOPPER_TYP_0.0,SHOPPER_TYP_1.0,SHOPPER_TYP_2.0,SHOPPER_TYP_3.0,VERS_TYP_1.0,VERS_TYP_2.0
0,-1.665871,-0.326023,-0.353063,-0.149514,-0.247607,-0.018874,-0.340189,-0.105655,-0.041983,0.864806,...,1.921718,0.495816,-0.131216,-0.108789,-0.407197,-0.521563,-0.425916,1.460445,1.147628,-0.811811
1,-1.665225,-0.326023,0.984863,-0.149514,-0.247607,-0.822656,1.151929,-0.105655,1.016662,1.369604,...,-0.520368,0.495816,-0.131216,-0.108789,-0.407197,-0.521563,2.347881,-0.684723,1.147628,-0.811811
2,-1.677546,4.700435,-0.353063,-0.149514,-0.247607,1.588691,-0.340189,-0.105655,1.016662,-1.659182,...,1.921718,0.495816,-0.131216,-0.108789,-0.407197,-0.521563,-0.425916,1.460445,1.147628,-0.811811
3,-1.677344,3.443821,-0.427392,-0.149514,-0.247607,-1.626438,-0.418721,-0.105655,1.016662,-1.659182,...,-0.520368,0.495816,-0.131216,-0.108789,-0.407197,-0.521563,-0.425916,1.460445,-0.871362,1.231815
4,-1.677021,-0.326023,-0.427392,-0.149514,-0.247607,1.588691,-0.418721,-0.105655,-0.041983,0.864806,...,-0.520368,0.495816,-0.131216,-0.108789,-0.407197,-0.521563,-0.425916,1.460445,1.147628,-0.811811


## Using the chosen model on the Test Data

In [35]:
# Rerun AdaBoostClassifier with optimized parameters
param_grid = {'algorithm': ['SAMME.R'],
             'learning_rate': [0.1],
             'n_estimators': [50]}

best_model = GridSearchCV(estimator=AdaBoostClassifier(), param_grid=param_grid, scoring='roc_auc', cv=5)
best_model.fit(X, y)

GridSearchCV(cv=5, error_score='raise',
       estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'algorithm': ['SAMME.R'], 'learning_rate': [0.1], 'n_estimators': [50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [36]:
# Calculate prediction probabilities for the test set and print the first 10 results
preds = best_model.predict_proba(mailout_test)
preds[:10]

array([[ 0.67973037,  0.32026963],
       [ 0.68142173,  0.31857827],
       [ 0.72811792,  0.27188208],
       [ 0.7412526 ,  0.2587474 ],
       [ 0.72096592,  0.27903408],
       [ 0.73650913,  0.26349087],
       [ 0.69849842,  0.30150158],
       [ 0.68216067,  0.31783933],
       [ 0.70345576,  0.29654424],
       [ 0.71108044,  0.28891956]])

Submitting to the Kaggle Competition

In [ ]:
# Generate a csv file to be submitted to Kaggle with the LNR id and prediction of response as columns.
# Use second column of the array as the probabilities to be entered (probability of a positive response).
# Print the first few lines of the kaggle dataframe.
kaggle = pd.DataFrame({'LNR':test_LNR.astype(np.int32), 'RESPONSE':preds[:, 1]})
kaggle.to_csv('kaggle.csv', index = False)
kaggle.head()